In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
!git clone https://github.com/fizyr/keras-retinanet.git

Cloning into 'keras-retinanet'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 5758 (delta 0), reused 0 (delta 0), pack-reused 5757
Receiving objects: 100% (5758/5758), 13.37 MiB | 224.00 KiB/s, done.
Resolving deltas: 100% (3865/3865), done.


In [3]:
%cd keras-retinanet/

!pip install .

/Users/tailtq/Code/machine-learning/kaggle/global-wheat-detection/keras-retinanet
Processing /Users/tailtq/Code/machine-learning/kaggle/global-wheat-detection/keras-retinanet
  Using cached https://files.pythonhosted.org/packages/ad/fd/6bfe87920d7f4fd475acd28500a42482b6b84479832bdc0fe9e589a60ceb/Keras-2.3.1-py2.py3-none-any.whl
     |████████████████████████████████| 61kB 755kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/71/e3/19762fdfc62877ae9102edf6342d71b28fbfd9dea3d2f96a882ce099b03f/Keras_Applications-1.0.8-py3-none-any.whl
  Created wheel for keras-retinanet: filename=keras_retinanet-0.5.1-cp37-cp37m-macosx_10_9_x86_64.whl size=126899 sha256=3d0bd8b9e196d5faa15c0390e05243cad998e801315ae6ae5a331c493a4dbd17
  Stored in directory: /Users/tailtq/Library/Caches/pip/wheels/cb/7b/fb/0d6e5096f0b3a107ac5b676d7952e4e100990c2def9d43d3b7
  Created wheel for keras-resnet: filename=keras_resnet-0.1.0-py2.py3-none-any.whl size=13344 sha256=0a5191f1b647842bd76fb3acdefa5a6

In [5]:
!python setup.py build_ext --inplace

running build_ext
cythoning keras_retinanet/utils/compute_overlap.pyx to keras_retinanet/utils/compute_overlap.c
/Users/tailtq/opt/anaconda3/lib/python3.7/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /Users/tailtq/Code/machine-learning/kaggle/global-wheat-detection/keras-retinanet/keras_retinanet/utils/compute_overlap.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'keras_retinanet.utils.compute_overlap' extension
creating build
creating build/temp.macosx-10.9-x86_64-3.7
creating build/temp.macosx-10.9-x86_64-3.7/keras_retinanet
creating build/temp.macosx-10.9-x86_64-3.7/keras_retinanet/utils
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/tailtq/opt/anaconda3/include -arch x86_64 -I/Users/tailtq/opt/anaconda3/include -arch x86_64 -I/Users/tailtq/opt/anaconda3/include/python3.7m -I/

In [14]:
%cd ..

/Users/tailtq/Code/machine-learning/kaggle/global-wheat-detection


In [6]:
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

In [15]:
root = "dataset/"
train_img = root+"train"
test_img = root+"test"
train_csv = root+"train.csv"
sample_submission = root+"sample_submission.csv"

In [16]:
train = pd.read_csv(train_csv)
train.head()

,image_id,width,height,bbox,source
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1
